# Analyzing factors that may affect electrification

In [ ]:
def clip_tif_file(tif_file:str) -> gpd.GeoDataFrame:

    """
    This function clips a tiff file into the boundary required (i.e. Rwanda) and returns a 
    geopandas dataframe.

    Input:
        - tif_file: String denoting path to tif file
    Returns:
        - geopandas.GeoDataFrame: GeoDataFrame containing the clipped raster data
    """

    with rasterio.open(tif_file) as src:
        # Read raster data and geometry
        array = src.read(1)
        transform = src.transform

        # Open the boundary shapefile
        boundary_gdf = gpd.read_file(get_admin_boundary("boundary")[0])

        # Make sure both GeoDataFrames have the same CRS
        boundary_gdf = boundary_gdf.to_crs(src.crs)

        # Create a bounding box that covers the extent of the raster
        xmin, ymin, xmax, ymax = src.bounds
        bbox = box(xmin, ymin, xmax, ymax)
        bbox_gdf = gpd.GeoDataFrame(geometry=[bbox], crs=src.crs)

        # Intersect the bounding box with the boundary shapefile
        intersection = gpd.overlay(boundary_gdf, bbox_gdf, how='intersection')

        # Clip the raster to the intersection geometry
        clipped, transform = mask(src, shapes=intersection.geometry, crop=True)

        # Create a GeoDataFrame directly from the clipped raster
        shapes_gen = rasterio.features.shapes(clipped, transform=transform)
        features = [{'geometry': geometry, 'properties': {'pixel_value': value}}
                    for (geometry, value) in shapes_gen]
        gdf_clipped = gpd.GeoDataFrame.from_features(features, crs=src.crs)

        gdf_clipped = gdf_clipped.to_crs(("EPSG:4326"))

    return gdf_clipped


In [ ]:
def compute_administrative_metric(gdf: gpd.GeoDataFrame, admin_level: str) -> gpd.GeoDataFrame:
    """
    This function takes in a geopandas dataframe of an index spread across pixels, 
    and localizes it to the region under study (i.e. sector, cell, or village).

    Inputs:
        - gdf: Geopandas Geodataframe containing the index being measured
        - admin_level: This is just a string showing if the amin level is 
            sector, cell, or village
        - admin_id: This is a string showing te unique identifier to an 
            admin region

    Returns:
        - gpd.GeoDataFrame with median calculations for 

    """

    # Get the file path and identifier for the specified admin level
    admin_path, admin_id = get_admin_boundary(admin_level=admin_level)

    # Read the admin shapefile
    admin_shp = gpd.read_file(admin_path)
    admin_shp = admin_shp.to_crs(("EPSG:4326"))

    # Perform intersection between admin shapefile and the given GeoDataFrame
    overlay = gpd.overlay(admin_shp, gdf, how="intersection")

    # Calculate median and retain the first geometry in case of multiple intersections
    index_summary = overlay.groupby([admin_id]).agg({
        "pixel_value": "median",
        "geometry": "first" 
    }).reset_index()


    # Index summary is joined to the initial admin_level file because by choosing first, it
    # only chooses the first index of gdf in the admin region and not the entire region
    index_summary = index_summary.loc[:, index_summary.columns != "geometry"].merge(
                        admin_shp[[admin_id, "geometry"]], on = f"{admin_id}",
                        suffixes=('_left', '_right'))
    
    index_summary = gpd.GeoDataFrame(index_summary, geometry="geometry", crs=admin_shp.crs)


    return index_summary

In [ ]:
df_merge_cell = pd.merge(cell_asset_wealth_2020[["Cell_ID", "pixel_value"]],
                    cell_median_consumption_2020, left_on="Cell_ID",
                    right_on="administra").drop(columns=["administra"])
df_merge_cell.columns = ["cell_id", "asset_wealth_index", "consumption_2020"]

In [ ]:
# Create scatterplot
sns.set_theme(style="ticks", palette="pastel")
sns.set(font="Verdana", font_scale=0.7,style="white")
sns.scatterplot(x='asset_wealth_index', y='consumption_2020', data=df_merge_cell, color = "steelblue", edgecolor = "black")
sns.despine();
plt.show()

In [ ]:
import plotly.graph_objects as go
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/1962_2006_walmart_store_openings.csv')
df.head()

data = []
layout = dict(
    title = 'New Walmart Stores per year 1962-2006<br>\
Source: <a href="http://www.econ.umn.edu/~holmes/data/WalMart/index.html">\
University of Minnesota</a>',
    # showlegend = False,
    autosize = False,
    width = 1000,
    height = 900,
    hovermode = False,
    legend = dict(
        x=0.7,
        y=-0.1,
        bgcolor="rgba(255, 255, 255, 0)",
        font = dict( size=11 ),
    )
)
years = df['YEAR'].unique()

for i in range(len(years)):
    geo_key = 'geo'+str(i+1) if i != 0 else 'geo'
    lons = list(df[ df['YEAR'] == years[i] ]['LON'])
    lats = list(df[ df['YEAR'] == years[i] ]['LAT'])
    # Walmart store data
    data.append(
        dict(
            type = 'scattergeo',
            showlegend=False,
            lon = lons,
            lat = lats,
            geo = geo_key,
            name = int(years[i]),
            marker = dict(
                color = "rgb(0, 0, 255)",
                opacity = 0.5
            )
        )
    )
    # Year markers
    data.append(
        dict(
            type = 'scattergeo',
            showlegend = False,
            lon = [-78],
            lat = [47],
            geo = geo_key,
            text = [years[i]],
            mode = 'text',
        )
    )
    layout[geo_key] = dict(
        scope = 'usa',
        showland = True,
        landcolor = 'rgb(229, 229, 229)',
        showcountries = False,
        domain = dict( x = [], y = [] ),
        subunitcolor = "rgb(255, 255, 255)",
    )


def draw_sparkline( domain, lataxis, lonaxis ):
    ''' Returns a sparkline layout object for geo coordinates  '''
    return dict(
        showland = False,
        showframe = False,
        showcountries = False,
        showcoastlines = False,
        domain = domain,
        lataxis = lataxis,
        lonaxis = lonaxis,
        bgcolor = 'rgba(255,200,200,0.0)'
    )

# Stores per year sparkline
layout['geo44'] = draw_sparkline({'x':[0.6,0.8], 'y':[0,0.15]}, \
                                 {'range':[-5.0, 30.0]}, {'range':[0.0, 40.0]} )
data.append(
    dict(
        type = 'scattergeo',
        mode = 'lines',
        lat = list(df.groupby(by=['YEAR']).count()['storenum']/1e1),
        lon = list(range(len(df.groupby(by=['YEAR']).count()['storenum']/1e1))),
        line = dict( color = "rgb(0, 0, 255)" ),
        name = "New stores per year<br>Peak of 178 stores per year in 1990",
        geo = 'geo44',
    )
)

# Cumulative sum sparkline
layout['geo45'] = draw_sparkline({'x':[0.8,1], 'y':[0,0.15]}, \
                                 {'range':[-5.0, 50.0]}, {'range':[0.0, 50.0]} )
data.append(
    dict(
        type = 'scattergeo',
        mode = 'lines',
        lat = list(df.groupby(by=['YEAR']).count().cumsum()['storenum']/1e2),
        lon = list(range(len(df.groupby(by=['YEAR']).count()['storenum']/1e1))),
        line = dict( color = "rgb(214, 39, 40)" ),
        name ="Cumulative sum<br>3176 stores total in 2006",
        geo = 'geo45',
    )
)

z = 0
COLS = 5
ROWS = 9
for y in reversed(range(ROWS)):
    for x in range(COLS):
        geo_key = 'geo'+str(z+1) if z != 0 else 'geo'
        layout[geo_key]['domain']['x'] = [float(x)/float(COLS), float(x+1)/float(COLS)]
        layout[geo_key]['domain']['y'] = [float(y)/float(ROWS), float(y+1)/float(ROWS)]
        z=z+1
        if z > 42:
            break

fig = go.Figure(data=data, layout=layout)
fig.update_layout(width=800)
fig.show()

In [ ]:
df_merge_sector_spending = merge_data_for_plots(sector_median_consumption_2020, spending_2020, "spending_index")
create_scatterplot(df_merge_sector_spending, "spending_index", "Spending")

In [ ]:
df_merge_sector_asset = merge_data_for_plots(sector_median_consumption_2020, sector_asset_wealth_2020, "asset_wealth_index")
create_scatterplot(df_merge_sector_asset, "asset_wealth_index", "Asset Wealth")

Asset Wealth

In [ ]:
# merge with asset wealth
village_asset_wealth_2016["date"] = "2016"
village_asset_wealth_2017["date"] = "2017"
village_asset_wealth_2018["date"] = "2018"
village_asset_wealth_2019["date"] = "2019"
village_asset_wealth_2020["date"] = "2020"

village_asset_wealth_2016.drop(columns = ["geometry"], inplace=True)
village_asset_wealth_2017.drop(columns = ["geometry"], inplace=True)
village_asset_wealth_2018.drop(columns = ["geometry"], inplace=True)
village_asset_wealth_2019.drop(columns = ["geometry"], inplace=True)
village_asset_wealth_2020.drop(columns = ["geometry"], inplace=True)

village_asset_wealth = pd.merge(pd.merge(pd.merge(pd.merge(village_asset_wealth_2016, 
                               village_asset_wealth_2017, on=["Village_ID", "date", "pixel_value"],how="outer"),
                               village_asset_wealth_2018, on=["Village_ID", "date", "pixel_value"],how="outer"),
                               village_asset_wealth_2019, on=["Village_ID", "date", "pixel_value"],how="outer"),
                               village_asset_wealth_2020, on=["Village_ID", "date", "pixel_value"],how="outer"
)
village_asset_wealth.columns = ["village_id", "asset_wealth", "year"]

village_asset_wealth["year"] = village_asset_wealth["year"].astype("str")

In [ ]:
final_df = pd.merge(final_df, village_asset_wealth, how="left", on = ["village_id", "year"])

Spending

In [ ]:
# merge with asset wealth
village_spending_2016["date"] = "2016"
village_spending_2017["date"] = "2017"
village_spending_2018["date"] = "2018"
village_spending_2019["date"] = "2019"
village_spending_2020["date"] = "2020"

village_spending_2016.drop(columns = ["geometry"], inplace=True)
village_spending_2017.drop(columns = ["geometry"], inplace=True)
village_spending_2018.drop(columns = ["geometry"], inplace=True)
village_spending_2019.drop(columns = ["geometry"], inplace=True)
village_spending_2020.drop(columns = ["geometry"], inplace=True)

village_spending = pd.merge(pd.merge(pd.merge(pd.merge(village_spending_2016, 
                               village_spending_2017, on=["Village_ID", "date", "pixel_value"],how="outer"),
                               village_spending_2018, on=["Village_ID", "date", "pixel_value"],how="outer"),
                               village_spending_2019, on=["Village_ID", "date", "pixel_value"],how="outer"),
                               village_spending_2020, on=["Village_ID", "date", "pixel_value"],how="outer"
)
village_spending.columns = ["village_id", "spending", "year"]

village_spending["year"] = village_spending["year"].astype("str")

In [ ]:
final_df = pd.merge(final_df, village_spending, how="left", on = ["village_id", "year"])

Population

In [ ]:
village_population_2012["date"] = "2012"
village_population_2013["date"] = "2013"
village_population_2014["date"] = "2014"
village_population_2015["date"] = "2015"
village_population_2016["date"] = "2016"
village_population_2017["date"] = "2017"
village_population_2018["date"] = "2018"
village_population_2019["date"] = "2019"
village_population_2020["date"] = "2020"

village_population_2012.drop(columns = ["geometry"], inplace=True)
village_population_2013.drop(columns = ["geometry"], inplace=True)
village_population_2014.drop(columns = ["geometry"], inplace=True)
village_population_2015.drop(columns = ["geometry"], inplace=True)
village_population_2016.drop(columns = ["geometry"], inplace=True)
village_population_2017.drop(columns = ["geometry"], inplace=True)
village_population_2018.drop(columns = ["geometry"], inplace=True)
village_population_2019.drop(columns = ["geometry"], inplace=True)
village_population_2020.drop(columns = ["geometry"], inplace=True)

village_population = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(village_population_2012, 
                               village_population_2013, on=["Village_ID", "date", "pixel_value"],how="outer"),
                               village_population_2014, on=["Village_ID", "date", "pixel_value"],how="outer"),
                               village_population_2015, on=["Village_ID", "date", "pixel_value"],how="outer"),
                               village_population_2016, on=["Village_ID", "date", "pixel_value"],how="outer"),
                               village_population_2017, on=["Village_ID", "date", "pixel_value"],how="outer"),
                               village_population_2018, on=["Village_ID", "date", "pixel_value"],how="outer"),
                               village_population_2019, on=["Village_ID", "date", "pixel_value"],how="outer"),
                               village_population_2020, on=["Village_ID", "date", "pixel_value"],how="outer"
)
village_population.columns = ["village_id", "population", "year"]

village_population["year"] = village_population["year"].astype("str")

In [ ]:
final_df = pd.merge(final_df, village_population, how="outer", on = ["village_id", "year"])

Urbanization

In [ ]:
village_urban_2010["date"] = "2010"
village_urban_2015["date"] = "2015"
village_urban_2020["date"] = "2020"

village_urban_2010.drop(columns = ["geometry"], inplace=True)
village_urban_2015.drop(columns = ["geometry"], inplace=True)
village_urban_2020.drop(columns = ["geometry"], inplace=True)

urbanization = pd.merge(pd.merge(village_urban_2010, 
                               village_urban_2015, on=["Village_ID", "date", "pixel_value"],how="outer"),
                               village_urban_2020, on=["Village_ID", "date", "pixel_value"],how="outer",
)
urbanization.columns = ["village_id", "urbanization", "year"]

urbanization["year"] = urbanization["year"].astype("str")

In [ ]:
final_df = pd.merge(final_df, urbanization, how="outer", on = ["village_id", "year"])

Building volume

In [ ]:
village_building_volume_2010["date"] = "2010"
village_building_volume_2015["date"] = "2015"
village_building_volume_2020["date"] = "2020"

village_building_volume_2010.drop(columns = ["geometry"], inplace=True)
village_building_volume_2015.drop(columns = ["geometry"], inplace=True)
village_building_volume_2020.drop(columns = ["geometry"], inplace=True)

building_volume = pd.merge(pd.merge(village_building_volume_2010, 
                               village_building_volume_2015, on=["Village_ID", "date", "pixel_value"],how="outer"),
                               village_building_volume_2020, on=["Village_ID", "date", "pixel_value"],how="outer",
)
building_volume.columns = ["village_id", "building_volume", "year"]

building_volume["year"] = building_volume["year"].astype("str")

In [ ]:
final_df = pd.merge(final_df, building_volume, how="outer", on = ["village_id", "year"])

Building Height

In [ ]:
village_building_height_2018.drop(columns = ["geometry"], inplace=True)
village_building_height_2018.columns = ["village_id", "building_height"]
final_df = pd.merge(final_df, village_building_height_2018, how="left", on = ["village_id"])

Landcover

In [ ]:
landcover = pd.read_csv("/gypsum/eguide/projects/ce8760/data/landcover/landcover.csv")
landcover["year"] = landcover["year"].astype(str)
landcover["village_id"] = landcover["village_id"].astype(str)

In [ ]:
landcover_2013 = pd.read_pickle("/gypsum/eguide/projects/ce8760/data/landcover/df_2013.pkl")
landcover_2014 = pd.read_pickle("/gypsum/eguide/projects/ce8760/data/landcover/df_2014.pkl")
landcover_2015 = pd.read_pickle("/gypsum/eguide/projects/ce8760/data/landcover/df_2015.pkl")
landcover_2016 = pd.read_pickle("/gypsum/eguide/projects/ce8760/data/landcover/df_2016.pkl")

In [ ]:
landcover_2013.columns = landcover.columns
landcover_2014.columns = landcover.columns
landcover_2015.columns = landcover.columns
landcover_2016.columns = landcover.columns

In [ ]:
landcover = pd.concat([landcover_2013, landcover_2014, landcover_2015, landcover_2016, landcover])

In [ ]:
landcover.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148150 entries, 0 to 88889
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   village_id            148150 non-null  object 
 1   cropland_proportion   148150 non-null  float64
 2   builtarea_proportion  148150 non-null  float64
 3   rangeland_proportion  148150 non-null  float64
 4   year                  148150 non-null  object 
dtypes: float64(3), object(2)
memory usage: 6.8+ MB


In [ ]:
landcover["year"] = landcover["year"].astype("str")
landcover["village_id"] = landcover["village_id"].astype("str")

In [ ]:
# final_df.drop(columns = ["cropland_proportion", "builtarea_proportion", "rangeland_proportion"], inplace = True)

In [ ]:
final_df["year"] = final_df["year"].astype("str")
final_df["village_id"] = final_df["village_id"].astype("str")

In [ ]:
final_df = pd.merge(final_df, landcover, how="outer", on = ["village_id", "year"])

In [ ]:
final_df

,village_id,year,distance_market,distance_busstation,distance_to_nearest_road,distance_schools,distance_banks,asset_wealth,spending,population,urbanization,building_volume,building_height,cropland_proportion,builtarea_proportion,rangeland_proportion
0,11010102,2012,0.988930,0.765208,0.248803,0.172175,0.443753,NaN,NaN,256.252640,NaN,NaN,5.085509,NaN,NaN,NaN
1,11010102,2013,0.988930,0.767398,0.248803,0.172175,0.439669,NaN,NaN,257.667900,NaN,NaN,5.085509,0.000000,1.000000,0.000000
2,11010102,2014,0.988848,0.765208,0.249891,0.173555,0.438713,NaN,NaN,265.239548,NaN,NaN,5.085509,0.000000,1.000000,0.000000
3,11010102,2015,0.988191,0.765208,0.249891,0.173555,0.442250,NaN,NaN,251.027534,30.0,50853.0,5.085509,0.000000,1.000000,0.000000
4,11010102,2016,0.988191,0.765208,0.249891,0.173555,0.442250,1.070581,6.413167,256.711594,NaN,NaN,5.085509,0.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177775,57150501,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.300282,0.158434,0.007784
177776,57150502,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.214361,0.196575,0.150272
177777,57150503,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.209166,0.133858,0.177291
177778,57150504,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.067908,0.430303,0.050098


In [ ]:
final_df.to_pickle("residential_customers_panel_10yr_above.pkl")

### Non-Residential Consumption

In [ ]:
nonresidential_annual_consumption_path = "../final_data/final_reg_data/annual_nonresidential_consumption_10yr_above/village/median/annual_median.csv"
noresidential_annual_tariff_path =  "../final_data/final_reg_data/annual_nonresidential_tariffs_10yr_above/village/median/annual_median.csv"
consumption_non_res_df = extract_consumption_tariffs(nonresidential_annual_consumption_path, "consumption")
tarrif_non_res_df = extract_consumption_tariffs(noresidential_annual_tariff_path, "tariff")

In [ ]:
consumption_non_res_df

,village_id,date,consumption
0,11010107,2012,123.2
1,11010107,2013,408.65
2,11010107,2014,462.25
3,11010107,2015,368.4
4,11010107,2016,368.9
...,...,...,...
3844,57100512,2016,229.6
3845,57100512,2017,217.7
3846,57100512,2018,193.65
3847,57100512,2019,309.5


In [ ]:
combined_data = pd.read_pickle("/gypsum/eguide/projects/ce8760/combined_data.pkl")

In [ ]:
combined_data = combined_data[combined_data["connection_type"] != "Residential"]
combined_data.connection_type.unique()

array(['other', 'Non Residential'], dtype=object)

In [ ]:
combined_data['transaction_date'] = pd.to_datetime(combined_data['transaction_date'], format='mixed')
combined_data['installation_date'] = pd.to_datetime(combined_data['installation_date'], format='mixed')

In [ ]:
# Calculate the earliest installation date per administrative region
earliest_dates = combined_data.groupby("village_id")['installation_date'].min()
combined_data = combined_data.merge(earliest_dates.rename('earliest_installation'), on="village_id")
combined_data = combined_data[combined_data['installation_date'] > combined_data['earliest_installation'] + pd.DateOffset(years=10)]
# combined_data = combined_data[(combined_data['installation_date'] > combined_data['earliest_installation'] + pd.DateOffset(years=5)) &
#         (combined_data['installation_date'] <= combined_data['earliest_installation'] + pd.DateOffset(years=10))]

In [ ]:
test_combined = combined_data.groupby([combined_data["meter_serial_number"], combined_data["connection_type"], 
                                       combined_data['transaction_date'].dt.year]).agg({
    "geometry": "first",
    "sector_id": "first",
    "cell_id": "first",
    "village_id": "first"
}).reset_index()

In [ ]:
meter_location = test_combined.groupby("meter_serial_number")[["geometry"]].first().reset_index()
meters_location = gpd.GeoDataFrame(meter_location, geometry="geometry", crs="EPSG:4326")

In [ ]:
# Assuming 'rwa_roads' is your GeoDataFrame containing the road geometries
road_points = []
for road in rwa_roads.geometry:
    road_points.extend(line_to_points(road, num_points=1000))  # Adjust num_points as needed

# Convert the list of Points into an array of [x, y] for cKDTree
road_points_array = np.array([[point.x, point.y] for point in road_points])

# Build the cKDTree for efficient nearest-neighbor queries
tree = cKDTree(road_points_array)

# Assuming 'meter_location' is your GeoDataFrame containing the meter point geometries
# Convert meter locations into an array of [x, y] for querying the cKDTree
meter_points_array = np.array([[point.x, point.y] for point in meter_location.geometry])

# Query the cKDTree for the nearest road point to each meter location
distances, indices = tree.query(meter_points_array, k=1)

distances_in_km = []

for distance in distances:
    distances_in_km.append(geodesic((0, 0), (0, distance)).kilometers)
    

# Add the distances as a new column to the meter_location GeoDataFrame
meter_location['distance_to_nearest_road'] = np.array(distances_in_km)

In [ ]:
test_combined = pd.merge(test_combined, meter_location[["meter_serial_number", "distance_to_nearest_road"]], 
                   how="left", on="meter_serial_number")

In [ ]:
village_distance_to_nearest_road = test_combined.groupby(["village_id", "transaction_date"]).agg({
    "distance_to_nearest_road": "median"
}).reset_index()

In [ ]:
# Create cKDTree using the shop geometries
tree = cKDTree(marketplace['geometry'].apply(lambda x: x.centroid.coords[0] if x.geom_type == 'Polygon' else x.coords[0]).tolist())

# Function to find the distance to the nearest shop
def find_nearest_shop_distance(row, tree):
    point = row['geometry'].coords[0]
    distance, _ = tree.query(point)

    # Convert distance from degrees to kilometers using Haversine formula
    distance_in_km = geodesic((0, 0), (0, distance)).kilometers
    
    return distance_in_km


# Apply the function to create a new 'distance' column in df1
test_combined['distance_market'] = test_combined.apply(lambda row: find_nearest_shop_distance(row, tree), axis=1)


In [ ]:
village_distance_to_market = test_combined.groupby(["village_id", "transaction_date"]).agg({
    "distance_market": "median"
}).reset_index()

In [ ]:
test_combined["distance_busstation"] = test_combined.apply(lambda row: find_nearest_shop_distance(row, bus_tree), axis=1)
# test_combined

In [ ]:
village_distance_to_busstation = test_combined.groupby(["village_id", "transaction_date"]).agg({
    "distance_busstation": "median"
}).reset_index()

In [ ]:
test_combined["distance_schools"] = test_combined.apply(lambda row: find_nearest_shop_distance(row, school_tree), axis=1)

village_distance_to_school = test_combined.groupby(["village_id", "transaction_date"]).agg({
    "distance_schools": "median"
}).reset_index()

In [ ]:
test_combined["distance_banks"] = test_combined.apply(lambda row: find_nearest_shop_distance(row, bank_tree), axis=1)

village_distance_to_banks = test_combined.groupby(["village_id", "transaction_date"]).agg({
    "distance_banks": "median"
}).reset_index()

In [ ]:
consumption_non_res_df = pd.merge(consumption_non_res_df, tarrif_non_res_df, right_on=["village_id", "date"], left_on=["village_id", "date"])

In [ ]:
consumption_non_res_df['date'] = consumption_non_res_df['date'].astype('object')
consumption_non_res_df['village_id'] = consumption_non_res_df['village_id'].astype('str')
tarrif_non_res_df['date'] = tarrif_non_res_df['date'].astype('object')
tarrif_non_res_df['village_id'] = tarrif_non_res_df['village_id'].astype('str')
village_distance_to_market['transaction_date'] = village_distance_to_market['transaction_date'].astype('str')
village_distance_to_busstation['transaction_date'] = village_distance_to_busstation['transaction_date'].astype('str')
village_distance_to_nearest_road['transaction_date'] = village_distance_to_nearest_road['transaction_date'].astype('str')
village_distance_to_school['transaction_date'] = village_distance_to_school['transaction_date'].astype('str')
village_distance_to_banks['transaction_date'] = village_distance_to_banks['transaction_date'].astype('str')

In [ ]:
final_non_res_df = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(consumption_non_res_df, 
                    village_distance_to_market, left_on=["village_id", "date"],
                    right_on=["village_id", "transaction_date"]).drop(columns=["transaction_date"]),
                    village_distance_to_busstation, left_on=["village_id", "date"],
                     right_on=["village_id", "transaction_date"]).drop(columns=["transaction_date"]),
    village_distance_to_nearest_road, left_on=["village_id", "date"],
                     right_on=["village_id", "transaction_date"]).drop(columns=["transaction_date"]),
    village_distance_to_school, left_on=["village_id", "date"],
                     right_on=["village_id", "transaction_date"]).drop(columns=["transaction_date"]),
    village_distance_to_banks, left_on=["village_id", "date"],
                     right_on=["village_id", "transaction_date"]).drop(columns=["transaction_date"])
                     

final_non_res_df = final_non_res_df.rename(columns={"date":"year"})
final_non_res_df["year"] = final_non_res_df["year"].astype("str")

In [ ]:
final_non_res_df

,village_id,year,consumption,tariff,distance_market,distance_busstation,distance_to_nearest_road,distance_schools,distance_banks
0,11010107,2012,123.2,16525.0,1.072830,0.518970,0.026218,0.488517,0.201774
1,11010107,2013,408.65,54871.5,1.072906,0.518970,0.026214,0.488534,0.201751
2,11010107,2014,462.25,62070.0,1.072906,0.518970,0.026214,0.488534,0.201751
3,11010107,2015,368.4,55941.5,1.072829,0.518942,0.026202,0.488617,0.201498
4,11010107,2016,368.9,67198.0,1.072828,0.518970,0.026211,0.488551,0.201729
...,...,...,...,...,...,...,...,...,...
3460,57100512,2016,229.6,42088.5,1.021832,0.461342,10.601017,1.002348,0.799732
3461,57100512,2017,217.7,41186.0,1.053419,0.621131,10.685755,0.961046,0.831267
3462,57100512,2018,193.65,37294.5,1.024816,0.621131,10.685755,1.000035,0.831267
3463,57100512,2019,309.5,58722.141,1.010919,0.461342,10.601017,1.006714,0.799732


In [ ]:
final_non_res_df = pd.merge(final_non_res_df, village_asset_wealth, how="left", on = ["village_id", "year"])
final_non_res_df = pd.merge(final_non_res_df, village_spending, how="left", on = ["village_id", "year"])
final_non_res_df = pd.merge(final_non_res_df, village_population, how="outer", on = ["village_id", "year"])
final_non_res_df = pd.merge(final_non_res_df, urbanization, how="outer", on = ["village_id", "year"])
final_non_res_df = pd.merge(final_non_res_df, building_volume, how="outer", on = ["village_id", "year"])
final_non_res_df = pd.merge(final_non_res_df, village_building_height_2018, how="left", on = ["village_id"])
final_non_res_df = pd.merge(final_non_res_df, landcover, how="outer", on = ["village_id", "year"])

In [ ]:
print("finish!")

finish!
